A notebook assist
-----------------

As you dig into the Susan Rice story, it's worth looking at [the article we posted from BuzzFeed on the role of one bot maker.](https://www.buzzfeed.com/josephbernstein/from-utah-with-love?utm_term=.aoMBYgagoB#.ckrmaXYXom) Now, to help you make searches, we give you this notebook with everything more or less in one place. 

** Pulling Tweets from Twitter**

First, load up your credetials!

In [ ]:
CONSUMER_KEY = "aCPFLPDed4n8ZSrUiZTg9wOSd"
CONSUMER_SECRET = "jR8SwCWUODcDG71yfijIa2BPvBTstGRwTWHQjvX3wouTZijH3e"
ACCESS_TOKEN = "2671-JdgE7uCzEdRKXZmCgtIOKa21catSxjCXMWvC8pzNQUzYR"
ACCESS_TOKEN_SECRET = "4QfZzyLOHOW0n29GLi3YP8LjpHBTpQGrhZ85a9cqUfQiX"

Now, we instantiate an instance of the API object from Tweepy that lets you easily make calls to Twitter's APIs. Recall the authentication object we needed to make first that encapsulates our credentials and gives us access to Twitter.

In [ ]:
# before we can make Twitter API calls, we need to initialize a few things...
from tweepy import OAuthHandler, API

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api = API(auth)

Now, we can make a search. Here we look for the hash tag of Susan Rice's name (it was promoted by the botster in the BuzzFeed article). We store the results in an object called `tweets` that behaves like a list. So, we can ask for tweets in position 0, 1, 10... The most recent tweets come first.

In [ ]:
tweets =api.search('#SusanRice', result_type='recent', count=100)

In [ ]:
tweets[0]

In [ ]:
type(tweets[0])

Remember, Tweepy has created a special object that represents a tweet. Handy! You get access to the attributes (data about the tweet) using familiar "dot" notation. Here's the text of the tweet, the number of times it was retweeted and the user's screen name.

In [ ]:
tweets[0].text

In [ ]:
tweets[0].retweet_count

In [ ]:
tweets[0].user.screen_name

(`screen_name` is stored inside the `user` attribute in a Tweepy status object.) We have seen that it is often useful to reformat this data into somethign more regular, like a DataFrame. Here we conduct a search, loop over the tweets, and pull out 3 kinds of information -- the text, retweet count and the poster's screen name...

In [ ]:
query = '#SusanRice'

texts = []
times  = []
retweets = []
screen_names = []

for tweet in api.search(query, result_type='recent', count=100):
    
    texts.append(tweet.text)
    times.append(tweet.created_at)
    retweets.append(tweet.retweet_count)
    screen_names.append(tweet.user.screen_name)

In [ ]:
retweets[:10]

In [ ]:
times[:10]

The last two cells show the first 10 values of retweet counts and tweet times. Notice that Tweepy has already turned the `created_at` field into a `datetime` object. Recall that this is Python's way of making times more expressive. Let's wrap all this into a DataFrame. We use a dictionary where the keys are column names and the associated values are lists that represent the values in each column.

In [ ]:
from pandas import DataFrame, to_datetime
df = DataFrame({"text":texts,"time":times,"retweet":retweets,"screen_name":screen_names})
df.head()

In [ ]:
df.tail()

Pulling 100 is great, but we can do more. The Twitter API let's us specify that our 100 tweets happen before a particular time (using `until`) or after a certain time (using `since`). You can also specify these ranges using tweet id's sice they are assigned consecutively. These arguments are `max_id` to specify pulling tweets before, and `since_id` to specify pulling tweets after the given id. 

Recall how we get help about something like `api.search`.

In [ ]:
?api.search

One last thing. If we want more than 100 tweets, we can use a Tweepy `Cursor`. It pushes back in time, looking for tweets, but it does all the accounting for you. You tell it how many tweets you want with each API call (100 max for search) and then how many calls (pages) you want to make. Below is the same code, but we ask for 150 API calls that work back in time, taking tweets in chunks of 100. If we are lucky, we'll get 100\*150 tweets.

(The way this works is that it makes an API call for 100. It then looks at the oldest tweet in the group and makes another call for 100 settig the `max_id` for the group we wat to be the id of the oldest tweet we have. So we go back in time from the last fetch. We just keep doing this with each new set of 100, repeating it 150 times.)

Oh and this will take a couple minutes.

In [ ]:
from tweepy import Cursor

query = '#SusanRice'

ids = []
texts = []
times  = []
retweets = []
screen_names = []
followers_counts = []
friends_counts = []

# Only iterate through the first 3 pages
for page in Cursor(api.search, q=query, result_type='recent', count=100).pages(150):
    
    for tweet in page:

        ids.append(tweet.id)
        texts.append(tweet.text)
        times.append(tweet.created_at)
        retweets.append(tweet.retweet_count)
        screen_names.append(tweet.user.screen_name)
        friends_counts.append(tweet.user.friends_count)
        followers_counts.append(tweet.user.followers_count)

... and make it a data frame. If you got an error indicating you've made too many API calls, don't worry. The code below will still work. You just won't have the full count of tweets you asked for. 

In [ ]:
from pandas import DataFrame
df = DataFrame({"id":ids,"text":texts,"time":times,"retweet":retweets,"screen_name":screen_names,"friends_count":friends_counts,"followers_count":followers_counts})
df.head()

In [ ]:
df.tail()

Next, let's plot like we did in the last notebook. We can make use of some cool grouping features in Pandas to create 10 minute activity plots (tweets with the hashtag #SusanRice, counted up in 10  minute intervals). For this  magic to work, we set the time to be an index. The `inplace=True` below means that we are making the change to the data frame `df` and not creating a new data frame with time as an index. 

We then use `TimeGrouper` to form groups in time and aggregate counts. We used `.agg()` before so we recall the subtle gymnastics to get the `counts` DataFrame in ship shape.

In [ ]:
df.set_index("time",inplace=True)
df.head()

In [ ]:
counts = df.groupby(TimeGrouper(freq='10min')).agg({"text":{"counts":"count"}})
counts.columns = counts.columns.droplevel()
counts = counts.reset_index()
counts.head()

And make a plot. The vertical access records the number of tweets with #SusanRice happening in a 10 minute period.

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

sign_in("cocteautt","9psj3t57ti")

mydata = [go.Scatter(x=counts["time"],y=counts["counts"],mode="line")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Notice this plot has jumps, sometimes quite abrupt. What happened?

Before we leave, this is how we restore the index of `df` back to 0,1,2.... and make `time` a column again.

In [ ]:
df.reset_index(inplace=True)
df.head()

If I want to scroll back farther, I can set my max_id to be the last ID I've collected so far. In my case that's as follows.

>`for page in Cursor(api.search, q=query, result_type='recent', count=5,max_id="850672995236544514").pages(1):`

Here's the code in one place.

In [ ]:
CONSUMER_KEY = "aCPFLPDed4n8ZSrUiZTg9wOSd"
CONSUMER_SECRET = "jR8SwCWUODcDG71yfijIa2BPvBTstGRwTWHQjvX3wouTZijH3e"
ACCESS_TOKEN = "2671-JdgE7uCzEdRKXZmCgtIOKa21catSxjCXMWvC8pzNQUzYR"
ACCESS_TOKEN_SECRET = "4QfZzyLOHOW0n29GLi3YP8LjpHBTpQGrhZ85a9cqUfQiX"

# before we can make Twitter API calls, we need to initialize a few things...
from tweepy import OAuthHandler, API, Cursor

# setup the authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api = API(auth)

query = '#SusanRice'

texts = []
ids = []
times  = []
retweets = []
screen_names = []
followers_counts = []
friends_counts = []

for page in Cursor(api.search, q=query, result_type='recent', count=100).pages(150):
    
    for tweet in page:

        ids.append(tweet.id)
        texts.append(tweet.text)
        times.append(tweet.created_at)
        retweets.append(tweet.retweet_count)
        screen_names.append(tweet.user.screen_name)
        friends_counts.append(tweet.user.friends_count)
        followers_counts.append(tweet.user.followers_count)
 
from pandas import DataFrame, to_datetime
df = DataFrame({"id":ids,"text":texts,"time":times,"retweet":retweets,"screen_name":screen_names,"friends_count":friends_counts,"followers_count":followers_counts})

We can store our tweets in files using the `.json` attribute of the Tweepy object. This is basically a dictionary/list Python version of the JSON string that Twitter returns. To write it out, we use a command `dumps` from the `json` package. It takes the kind of dictionary and list representation in Python and outputs a JSON string that represents the object.

Here'show Mike writes out a tweet. First have a look at the raw JSON and in the second cell, print it out to a file.

In [ ]:
tweet._json

In [ ]:
from json import dumps

out = open(str(tweet.id)+".json","wb")
out.write(dumps(tweet._json))
out.close()

Use the finder and have a look the file. It should resemble the files Mike gave you Thursday. 

**Reading Tweets from JSON Files**

And about those files... Below we assume you have unzipped the `thegatewaypundit_tweets.tar.gz` file and put it in the same folder as this notebook. Here we use `loads` from the `json` package -- it is the inverse of `dumps`. It takes a string and makes a Python object out of it. We use the `listdir` command from the `os` (for operating system) package to list the contents of my folder -- what JSON files did we get?

Oh and recall that these were obtained by doing a search for the [Gateway Pundit article on Hannity and Prince being unmasked.](http://www.thegatewaypundit.com/2017/04/breaking-rice-gate-obama-admin-spied-sean-hannity-erik-prince-video/) The other folder we pointed you to in email had to do with #rice-gate.

In [ ]:
from os import listdir
from json import loads

folder = "thegatewaypundit_tweets/"
tweet_folder = listdir(folder)
len(tweet_folder)

So 2,954 files (each a tweet mentioning the Hannity claim published in The Gateway Pundit. It's just a list of strings that are file names. Here are a few.

In [ ]:
tweet_folder[:10]

And here is how we open any one of them. We turn the string content of the file into a Python object using `loads`. (We include `folder+` in the `open()` command because these files are in the gatewaypunit folder.

In [ ]:
tweet = loads(open(folder+'849313754152148992.json').read())

In [ ]:
type(tweet)

In [ ]:
tweet.keys()

See? This technique makes a tweet into a Python dictionary with keys that represet the text of the tweet, the retweet_count,and facts about the user, stored in a second dictionary.

In [ ]:
tweet["text"]

In [ ]:
tweet["source"]

In [ ]:
tweet["retweet_count"]

In [ ]:
tweet["user"]["screen_name"]

This is how we turn the set of JSON files into a DataFrame. We loop over the files in the folder below and just pring out some facts about each tweet, or the first 100 tweets.

In [ ]:
for tweet_file in tweet_folder[:100]:
    
    tweet = loads(open(folder+tweet_file).read())
    print tweet["text"]
    print tweet["created_at"]
    print tweet["user"]["screen_name"]
    print tweet["user"]["followers_count"]
    print tweet["user"]["friends_count"]
    print "---"

So, let's do as before and store up these details so we can load them into a data frame.

In [ ]:
ids = []
texts = []
times = []
retweets = []
friends = []
followers = []
screen_names = []

for tweet_file in tweet_folder:
    
    tweet = loads(open(folder+tweet_file).read())
    
    ids.append(tweet["id"])
    texts.append(tweet["text"])
    times.append(tweet["created_at"])
    screen_names.append(tweet["user"]["screen_name"])
    followers.append(tweet["user"]["followers_count"])
    friends.append(tweet["user"]["friends_count"])
    retweets.append(tweet["retweet_count"])

In turning the data into columns of a data frame, we need to do one extra thing. We need to turn Twitter's `created_at` string into a Python datetime object. Tweepy was doing that for us. But now we're on our own. The `to_datetime` function from Pandas to the rescue!

In [ ]:
from pandas import DataFrame, to_datetime, TimeGrouper

df = DataFrame({"id":ids,"text":texts,"time":to_datetime(times),"screen_name":screen_names,"followers":followers,"friends":friends,"retweet":retweets})
df.head()

The files in our folder are sorted in increasing order of the tweet id. Our other data frame (pulled from Twitter) was organized in decreasing order, with the newest tweets at the top. Just something to keep in mind. We could sort the table if we wanted to make them consistent.

And now plot as we did before.

In [ ]:
df.set_index("time",inplace=True)
df.head()

In [ ]:
counts = df.groupby(TimeGrouper(freq='10min')).agg({"text":{"counts":"count"}})
counts.columns = counts.columns.droplevel()
counts = counts.reset_index()
counts.head()

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

sign_in("cocteautt","9psj3t57ti")

mydata = [go.Scatter(x=counts["time"],y=counts["counts"],mode="line")]
mylayout = go.Layout(autosize=False, width=1000,height=500)
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Reset the index on the data frame.

In [ ]:
df.reset_index(inplace=True)
df.head()

With this branch of the story, we can see the Gateway Pundit published it second (who scooped them?) and then it jumped.  Here's who was involved. Now you take up the rest!

In [ ]:
df["screen"].value_counts()